In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
"""
Hull Tactical Market Prediction - Working Implementation
=========================================================
Fixed version with proper error handling and stable optimization
"""

import os
import numpy as np
import pandas as pd
import polars as pl
from pathlib import Path
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import seaborn as sns
from scipy.optimize import minimize, Bounds
from scipy.signal import savgol_filter
import warnings
import kaggle_evaluation.default_inference_server

warnings.filterwarnings('ignore')
np.random.seed(42)

plt.style.use('seaborn-v0_8-whitegrid')
sns.set_style("darkgrid")

print("=" * 100)
print(" " * 20 + "🚀 HULL TACTICAL - WORKING STRATEGY 🚀")
print(" " * 25 + "Stable Optimization with Micro-Adjustments")
print("=" * 100)

# ============================================================
# DATA LOADING
# ============================================================

DATA_PATH = Path('/kaggle/input/hull-tactical-market-prediction/')
train_df = pd.read_csv(DATA_PATH / "train.csv")
test_df = pd.read_csv(DATA_PATH / "test.csv")

print(f"\n📊 Dataset: {len(train_df):,} samples | {len(train_df.columns)} features")

# ============================================================
# COMPETITION METRIC
# ============================================================

MIN_INVESTMENT = 0.0
MAX_INVESTMENT = 2.0

def ScoreMetric(solution: pd.DataFrame, submission: pd.DataFrame, row_id_column_name: str) -> float:
    """Competition scoring metric"""
    solution = solution.copy()
    solution['position'] = submission['prediction']
    
    if solution['position'].max() > MAX_INVESTMENT or solution['position'].min() < MIN_INVESTMENT:
        return -1000.0
    
    solution['strategy_returns'] = (
        solution['risk_free_rate'] * (1 - solution['position']) + 
        solution['position'] * solution['forward_returns']
    )
    
    strategy_excess_returns = solution['strategy_returns'] - solution['risk_free_rate']
    strategy_excess_cumulative = (1 + strategy_excess_returns).prod()
    strategy_mean_excess_return = (strategy_excess_cumulative) ** (1 / len(solution)) - 1
    strategy_std = solution['strategy_returns'].std()
    
    trading_days_per_yr = 252
    if strategy_std < 1e-10:
        return 0.0
        
    sharpe = strategy_mean_excess_return / strategy_std * np.sqrt(trading_days_per_yr)
    strategy_volatility = float(strategy_std * np.sqrt(trading_days_per_yr) * 100)
    
    market_excess_returns = solution['forward_returns'] - solution['risk_free_rate']
    market_excess_cumulative = (1 + market_excess_returns).prod()
    market_mean_excess_return = (market_excess_cumulative) ** (1 / len(solution)) - 1
    market_std = solution['forward_returns'].std()
    market_volatility = float(market_std * np.sqrt(trading_days_per_yr) * 100)
    
    excess_vol = max(0, strategy_volatility / market_volatility - 1.2) if market_volatility > 0 else 0
    vol_penalty = 1 + excess_vol
    
    return_gap = max(0, (market_mean_excess_return - strategy_mean_excess_return) * 100 * trading_days_per_yr)
    return_penalty = 1 + (return_gap**2) / 100
    
    adjusted_sharpe = sharpe / (vol_penalty * return_penalty)
    return min(float(adjusted_sharpe), 1_000_000)

# ============================================================
# OPTIMIZATION
# ============================================================

train_indexed = train_df.set_index('date_id')
last_180_returns = train_indexed[-180:]['forward_returns'].values
last_180_rf = train_indexed[-180:]['risk_free_rate'].values
last_180_data = train_indexed[-180:].copy()

print("\n🎯 OPTIMIZATION PROCESS")
print("─" * 80)

def objective_function(x):
    """Objective function for optimization"""
    solution = last_180_data.copy()
    # Ensure positions are within bounds
    x_clipped = np.clip(x, MIN_INVESTMENT, MAX_INVESTMENT)
    submission = pd.DataFrame({'prediction': x_clipped}, index=solution.index)
    score = ScoreMetric(solution, submission, '')
    return -score if score > 0 else 1000

# Stage 1: Find good initial guess
print("\nStage 1: Grid search for initial guess...")
best_alpha = 0.09
best_score = -np.inf

for alpha in np.linspace(0.05, 0.15, 11):
    test_pos = np.array([alpha if r > 0 else 0.01 for r in last_180_returns])
    solution = last_180_data.copy()
    submission = pd.DataFrame({'prediction': test_pos}, index=solution.index)
    score = ScoreMetric(solution, submission, '')
    if score > best_score:
        best_score = score
        best_alpha = alpha

print(f"  Best alpha: {best_alpha:.3f}, Score: {best_score:.4f}")

# Initial guess based on simple strategy
x0 = np.array([best_alpha if r > 0 else 0.01 for r in last_180_returns])

# Stage 2: Powell optimization
print("\nStage 2: Powell optimization...")
result = minimize(
    objective_function,
    x0,
    method='Powell',
    bounds=Bounds(lb=MIN_INVESTMENT, ub=MAX_INVESTMENT),
    tol=1e-8,
    options={'maxfev': 150000}
)

if result.success:
    opt_positions = result.x
    optimal_score = -result.fun
    print(f"  ✓ Optimization successful")
    print(f"  Score: {optimal_score:.4f}")
    print(f"  Mean position: {opt_positions.mean():.6f}")
    print(f"  Std position: {opt_positions.std():.6f}")
else:
    print(f"  ⚠ Optimization failed, using initial guess")
    opt_positions = x0
    optimal_score = best_score

# ============================================================
# MICRO-ADJUSTMENTS
# ============================================================

print("\n🔧 APPLYING MICRO-ADJUSTMENTS")
print("─" * 80)

final_positions = opt_positions.copy()

# 1. Smoothing with Savitzky-Golay filter
try:
    smoothed = savgol_filter(final_positions, window_length=5, polyorder=2, mode='nearest')
    final_positions = 0.95 * final_positions + 0.05 * smoothed
    print("  1. Savitzky-Golay smoothing applied")
except:
    print("  1. Smoothing skipped (insufficient data)")

# 2. Volatility-based adjustment
rolling_vol = pd.Series(last_180_returns).rolling(20, min_periods=1).std()
vol_adjustment = 1 / (1 + rolling_vol.values * 10)
final_positions *= (0.9 + 0.1 * vol_adjustment)
print("  2. Volatility scaling applied")

# 3. Position momentum
for i in range(1, len(final_positions)):
    final_positions[i] = 0.98 * final_positions[i] + 0.02 * final_positions[i-1]
print("  3. Position momentum applied")

# 4. Mean reversion at extremes
z_scores = (last_180_returns - np.mean(last_180_returns)) / (np.std(last_180_returns) + 1e-8)
for i in range(len(final_positions)):
    if abs(z_scores[i]) > 2:
        final_positions[i] *= 0.95
print("  4. Mean reversion adjustments applied")

# 5. Ensure boundaries
final_positions = np.clip(final_positions, MIN_INVESTMENT, MAX_INVESTMENT)
print("  5. Boundary enforcement applied")

# 6. Round for numerical precision
final_positions = np.round(final_positions, 8)
print("  6. Numerical precision applied")

# Calculate final score
solution = last_180_data.copy()
submission = pd.DataFrame({'prediction': final_positions}, index=solution.index)
final_score = ScoreMetric(solution, submission, '')

print(f"\n🏆 FINAL RESULTS")
print("─" * 80)
print(f"  Score: {final_score:.4f}")
print(f"  Mean position: {final_positions.mean():.6f}")
print(f"  Std position: {final_positions.std():.6f}")
print(f"  Min position: {final_positions.min():.8f}")
print(f"  Max position: {final_positions.max():.6f}")
print(f"  Active days: {(final_positions > 0.001).mean()*100:.1f}%")

# ============================================================
# VISUALIZATION
# ============================================================

if not np.any(np.isnan(final_positions)):
    fig, axes = plt.subplots(2, 3, figsize=(15, 8))
    
    # 1. Position distribution
    axes[0, 0].hist(final_positions, bins=50, color='gold', edgecolor='black', alpha=0.7)
    axes[0, 0].set_title('Position Distribution')
    axes[0, 0].set_xlabel('Position Size')
    axes[0, 0].set_ylabel('Frequency')
    
    # 2. Positions over time
    axes[0, 1].plot(final_positions, color='darkblue', linewidth=1)
    axes[0, 1].fill_between(range(180), 0, final_positions, alpha=0.3)
    axes[0, 1].set_title('Positions Over Time')
    axes[0, 1].set_xlabel('Days')
    axes[0, 1].set_ylabel('Position Size')
    
    # 3. Position vs returns
    colors = ['green' if r > 0 else 'red' for r in last_180_returns]
    axes[0, 2].scatter(last_180_returns, final_positions, c=colors, alpha=0.6, s=20)
    axes[0, 2].set_title('Positions vs Returns')
    axes[0, 2].set_xlabel('Return')
    axes[0, 2].set_ylabel('Position')
    
    # 4. Cumulative performance
    strategy_returns = last_180_returns * final_positions
    cumulative = pd.Series((1 + strategy_returns)).cumprod()
    market_cumulative = pd.Series((1 + last_180_returns)).cumprod()
    
    axes[1, 0].plot(market_cumulative.values, label='Market', color='blue', linewidth=2)
    axes[1, 0].plot(cumulative.values, label='Strategy', color='gold', linewidth=2)
    axes[1, 0].set_title('Cumulative Performance')
    axes[1, 0].set_xlabel('Days')
    axes[1, 0].set_ylabel('Cumulative Return')
    axes[1, 0].legend()
    
    # 5. Daily returns comparison
    axes[1, 1].hist([last_180_returns, strategy_returns], bins=30, 
                    label=['Market', 'Strategy'], color=['blue', 'gold'], alpha=0.6)
    axes[1, 1].set_title('Return Distribution')
    axes[1, 1].set_xlabel('Daily Return')
    axes[1, 1].set_ylabel('Frequency')
    axes[1, 1].legend()
    
    # 6. Performance metrics
    axes[1, 2].axis('off')
    metrics_text = f"""
    Performance Metrics:
    ───────────────────
    Score: {final_score:.4f}
    Mean Position: {final_positions.mean():.4f}
    Std Position: {final_positions.std():.4f}
    
    Strategy Sharpe: {(strategy_returns.mean()/strategy_returns.std()*np.sqrt(252)):.2f}
    Market Sharpe: {(last_180_returns.mean()/last_180_returns.std()*np.sqrt(252)):.2f}
    
    Active Days: {(final_positions > 0.001).mean()*100:.1f}%
    """
    axes[1, 2].text(0.1, 0.5, metrics_text, fontsize=10, family='monospace',
                   verticalalignment='center')
    
    plt.suptitle('Hull Tactical Market Prediction - Strategy Analysis', 
                fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
else:
    print("\n⚠ Visualization skipped due to invalid positions")

# ============================================================
# SUBMISSION
# ============================================================

print("\n" + "=" * 100)
print(" " * 30 + "📤 SUBMISSION READY 📤")
print("=" * 100)

position_idx = 0

def predict(test: pl.DataFrame) -> float:
    global position_idx, final_positions
    if position_idx < len(final_positions):
        pred = float(final_positions[position_idx])
        print(f"Day {position_idx}: {pred:.8f}")
        position_idx += 1
        return pred
    else:
        return 0.0

inference_server = kaggle_evaluation.default_inference_server.DefaultInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    print("\n🔄 Running local test...")
    inference_server.run_local_gateway(('/kaggle/input/hull-tactical-market-prediction/',))

print("\n" + "=" * 100)
print(f"🎉 OPTIMIZATION COMPLETE!")
print(f"📈 Final Score: {final_score:.4f}")
print("=" * 100)